In [12]:
import psycopg2
import boto3
import configparser
import csv

In [2]:
postgres_parser = configparser.ConfigParser()

In [3]:
postgres_parser.read("pipeline.conf")

['pipeline.conf']

In [4]:
host = "localhost"
port = postgres_parser.get("postgres_config" , "port")
username = postgres_parser.get("postgres_config" , "username")
password = postgres_parser.get("postgres_config" , "password")
db = postgres_parser.get("postgres_config" , "database")

In [7]:
conn = psycopg2.connect(database = db , host=host , port=port , user=username , password=password)

In [8]:
query = """ select * from customers"""
cursor = conn.cursor()

In [9]:
cursor.execute(query)

In [10]:
result = cursor.fetchall()

In [11]:
result

[(100, 'Jane', 'USA'), (101, 'Bob', 'UK'), (102, 'Miles', 'UK')]

In [13]:
filename = "customers.csv"

In [14]:
with open(filename ,'w') as f:
    writer = csv.writer(f , delimiter='|')
    writer.writerows(result)

In [15]:
cursor.close()
conn.close()

In [16]:
# load data to s3
aws_parser = configparser.ConfigParser()
aws_parser.read("pipeline.conf")

['pipeline.conf']

In [17]:
access_key = aws_parser.get("aws_boto_credentials","access_key")
secret_key = aws_parser.get("aws_boto_credentials","secret_key")
bucket_name = aws_parser.get("aws_boto_credentials","bucket_name")

In [18]:
s3 = boto3.client('s3' , 
                 aws_access_key_id = access_key,
                aws_secret_access_key = secret_key)

In [19]:
s3.upload_file(
filename,
bucket_name,
filename)